# Required packages
Please make sure you have ggplot2 and dplyr installed. To do so, go into R and enter the following command: <br>
`install.packages(c('ggplot2','dplyr'))` <br>
This can also be done using Anaconda: <br>
`conda install -c r r-dplyr` <br>
`conda install -c r r-ggplot2 
`

# Load [IBD GWAS summary stats](ftp://ftp.sanger.ac.uk/pub/consortia/ibdgenetics/iibdgc-trans-ancestry-filtered-summary-stats.tgz)

We will be working with GWAS summary statistics from the [International Inflammatory Bowel Disease Genetics Consortium (IIBDGC)](https://www.ibdgenetics.org/about.html). There are two main types of inflammatory bowel disease (IBD): Crohn's disease (CD) and ulcerative colitis (UC). <br> 

### Associated references
1. Liu, J.Z., van Sommeren, S., Huang, H., Ng, S.C., Alberts, R., Takahashi, A., Ripke, S., Lee, J.C., Jostins, L., Shah, T. and Abedian, S., 2015. Association analyses identify 38 susceptibility loci for inflammatory bowel disease and highlight shared genetic risk across populations. Nature genetics, 47(9), pp.979-986. Vancouver
2. Goyette, P., Boucher, G., Mallon, D., Ellinghaus, E., Jostins, L., Huang, H., Ripke, S., Gusareva, E.S., Annese, V., Hauser, S.L. and Oksenberg, J.R., 2015. High-density mapping of the MHC identifies a shared role for HLA-DRB1 [ast] 01: 03 in inflammatory bowel diseases and heterozygous advantage in ulcerative colitis. Nature genetics, 47(2), pp.172-179. Vancouver

We will be working with a thinned dataset (1% of rows randomly selected) for practicality. 

In [ ]:
suppressMessages(library(ggplot2))
suppressMessages(library(dplyr))

In [ ]:
ibd <- read.table('IBD_GWAS_summary_thinned.txt',header=TRUE)

In [ ]:
head(ibd)

To start, let's a take a look at the distribution of p-values for the SNPs in the summary statistics:

In [ ]:
hist(ibd$P, breaks=50)

Though the distribution is fairly uniform, there's a relatively large number of SNPs with low (significant) p-values! We can get a better idea understanding of what's going on with the lower tail with a qq-plot, which reflects the deviation of a distribution from the null expected distribution. <br>

"The quantile-quantile (qq) plot is a graphical technique for determining if two data sets come from populations with a common distribution.
A qq plot is a plot of the quantiles of the first data set against the quantiles of the second data set. By a quantile, we mean the fraction (or percent) of points below the given value. That is, the 0.3 (or 30%) quantile is the point at which 30% percent of the data fall below and 70% fall above that value.

A 45-degree reference line is also plotted. If the two sets come from a population with the same distribution, the points should fall approximately along this reference line. The greater the departure from this reference line, the greater the evidence for the conclusion that the two data sets have come from populations with different distributions."

The advantages of the q-q plot are:

1. The sample sizes do not need to be equal.
1. Many distributional aspects can be simultaneously tested. For example, shifts in location, shifts in scale, changes in symmetry, and the presence of outliers can all be detected from this plot. For example, if the two data sets come from populations whose distributions differ only by a shift in location, the points should lie along a straight line that is displaced either up or down from the 45-degree reference line.

([source](https://www.itl.nist.gov/div898/handbook/eda/section3/qqplot.htm))

# Plot QQ-plot

In [ ]:
qq_df <- data.frame(observed=-log10(sort(ibd$P)),expected=-log10(ppoints(length(ibd$P))))

log10Pe <- expression(paste("Expected -log"[10], "p"))
log10Po <- expression(paste("Observed -log"[10], "p"))

ggplot(qq_df) + 
    geom_point(aes(expected, observed), shape = 1, size = 3) +
    geom_abline(intercept = 0, slope = 1, alpha = 0.5, color='blue') +
    xlab(log10Pe) + ylab(log10Po)

Here, we use $-\log_{10} p$. SNPs with large $p$ have smaller $-\log_{10}p$, and they appear to largely follow the null expected (the diagonal line). They quickly began to deviate from the null distribution as $p$ gets smaller ($-\log_{10}p$ gets larger) and more significant. Thus, we have high confidence that there are genetic loci with strong statistical associations with IBD.

Next, let's visualize the GWAS results.

# Manhattan plot
## 1. Calculate cumulative position of each SNP

In [ ]:
plt_data <- ibd %>% 
  
  # Compute chromosome size
  group_by(CHR) %>% 
  summarise(chr_len=max(BP)) %>% # get largest BP value for each chr; makes chr_len column
  
  # Calculate cumulative position of each chromosome
  mutate(tot=cumsum(chr_len)-chr_len) %>% # makes tot column
  select(-chr_len) %>% # remove chr_len column; tot column remains
  
  # Add this info to the initial dataset
  left_join(ibd, ., by=c("CHR"="CHR")) %>% # add tot column by matching to chromosome
  
  # Add a cumulative position of each SNP
  arrange(CHR, BP) %>% #arrange by bp
  mutate( BPcum=BP+tot) #add BPcum column

In [ ]:
head(plt_data)

## 2. Prepare x-axis

In [ ]:
# give location of axis tick for chromosome centered on window for each chr
axisdf <- plt_data %>% group_by(CHR) %>% summarize(center=( max(BPcum) + min(BPcum) ) / 2 )

## 3. Plot

In [ ]:
ggplot(plt_data, aes(x=BPcum, y=-log10(P))) +
    
    # Show all points
    geom_point( aes(color=as.factor(CHR)), size=1.3) +
    scale_color_manual(values = rep(c("goldenrod1", "skyblue"), 22 )) +
    
    # custom X axis:
    scale_x_continuous( label = axisdf$CHR, breaks= axisdf$center ) +
    #scale_y_continuous(expand = c(0, 0) ) +     # remove space between plot area and x axis
  
    # Custom theme:
    theme_bw() +
    theme( 
      legend.position="none",
      panel.border = element_blank(),
      panel.grid.major.x = element_blank(),
      panel.grid.minor.x = element_blank()) +

    # Add line for genome-wide significance
    geom_hline(yintercept = -log10(5*10^-8), color='red') + 
    # Rename x-axis
    labs(x='Chr',y=expression(paste("-log"[10], "p")))

We added a red line to denote where $p=10^{-8}$. This is the widely-used __genome-wide significance__ threshold. Points which fall above the red line in the Manhattan plot have genome-wide signficance! It looks like there's a very defined peak of genome-wide significant SNPs localized on chromosome 6. Let's take a closer look. <br>

*By the way, why do you think Manhattan plots have so many peaks like this one?*

In [ ]:
chr6_gws <- ibd %>% filter(CHR == 6, P < 10^(-8))
sprintf("# of genome-wide significant SNPs in (thinned) IBD GWAS: %d", dim(chr6_gws)[1])

What are the positions of the genome-wide significant SNPs on chr6?

In [ ]:
sort(chr6_gws$BP)

Let's take a closer look at what that region is: <br>
<img src="mhc_genome_browser_hg19.png">

What's so special about this region? There is a high density of __human leukocyte antigen (HLA)__ genes here - these encode the __major histocompatibility complex (MHC)__ proteins in humans. These proteins are essential for the immune system to recognize foreign molecules. It is a very polygenic region, as diversity of antigen presentation is advantageous for the effectiveness of the immune system for targeting foreign invaders. This corroborates the findings of the paper based on this GWAS data (Goyette P., et al. Nature 2015). The MHC region is often implicated in autoimmune like IBD, and many other complex diseases. You can read more [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5406920/) if you're interested.